In [40]:
from typing import List, Dict

def preprocessing(load, fuels, powerplants):
    # Add cost to powerplants
    for powerplant in powerplants:
        powerplant["cost"] = \
            (powerplant["type"] == "windturbine") * 0 + \
            (powerplant["type"] == "gasfired") * fuels["gas(euro/MWh)"] + \
            (powerplant["type"] == "turbojet") * fuels["kerosine(euro/MWh)"]
    
    # Reduce windturbines pmin and pmax because of wind
    for powerplant in powerplants:
        if powerplant["type"] == "windturbine":
            powerplant["pmax"] = round(powerplant["pmax"] * fuels["wind(%)"] / 100, 1)
    return load, fuels, powerplants

def postprocessing(production_plan):
    for powerplant in production_plan:
        powerplant["p"] = round(float(powerplant["p"]), 1)
    return production_plan

def calculate_production_plan(load, fuels, powerplants):
    # Sort powerplants based on cost, efficiency, pmin and pmax
    sorted_powerplants = sorted(powerplants, key=lambda p: (p["cost"], \
                                                            p["efficiency"], \
                                                            p["pmin"], \
                                                            -p["pmax"]))
    print(json.dumps(sorted_powerplants, sort_keys=True, indent=4))
    # Initialize variables
    remaining_load = load
    production_plan = []

    # Iterate over powerplants and allocate power based on merit-order
    for powerplant in sorted_powerplants:
        pmax = powerplant["pmax"]
        pmin = powerplant["pmin"]

        # Calculate the maximum power that can be allocated for the powerplant
        max_power = min(remaining_load, pmax)

        # Calculate the power to be allocated while considering pmin
        allocated_power = max(max_power, pmin)

        # Update the remaining load and power production plan
        remaining_load -= allocated_power
        production_plan.append({"name": powerplant["name"],"p": allocated_power})
        
        # Keep the last powerplant in a variable for later
        last_powerplant = powerplant
        
        # Break the loop if the remaining load is either 0 ()
        if remaining_load <= 0:
            break
    if remaining_load == 0 :
        return production_plan
    
    else: # when remaining_load > 0 because the pmin of the last powerplant is too high
        first_powerplants = powerplants.copy()
        first_powerplants.remove(last_powerplant)
        load4first_powerplants = load - last_powerplant["pmin"]
        
        print("last_powerplant : ")
        print(json.dumps(last_powerplant, sort_keys=True, indent=4))
        print('-----------------------------------------------------------------------')
        # we re-calculate the production plan but we force the last powerplant to be added with its pmin as power.
        return [{"name": last_powerplant["name"],"p": last_powerplant["pmin"]}] + \
                calculate_production_plan(powerplants = first_powerplants,\
                                          load = load4first_powerplants,\
                                          fuels = fuels)


In [41]:
import json
with open('../tests/payloads/payload2.json', 'r') as f:
    data = json.load(f)
    #print(get_checks(data))
    load, fuels, powerplants = data['load'], data['fuels'], data['powerplants']
    load, fuels, powerplants = preprocessing(load, fuels, powerplants)
    production_plan = calculate_production_plan(load, fuels, powerplants)
    production_plan = postprocessing(production_plan)
    print(json.dumps(production_plan, sort_keys=True, indent=4))
    print(load)


load : 
480
sorted_powerplants : 
[
    {
        "cost": 0.0,
        "efficiency": 1,
        "name": "windpark1",
        "pmax": 0.0,
        "pmin": 0,
        "type": "windturbine"
    },
    {
        "cost": 0.0,
        "efficiency": 1,
        "name": "windpark2",
        "pmax": 0.0,
        "pmin": 0,
        "type": "windturbine"
    },
    {
        "cost": 13.4,
        "efficiency": 0.37,
        "name": "gasfiredsomewhatsmaller",
        "pmax": 210,
        "pmin": 40,
        "type": "gasfired"
    },
    {
        "cost": 13.4,
        "efficiency": 0.53,
        "name": "gasfiredbig1",
        "pmax": 460,
        "pmin": 100,
        "type": "gasfired"
    },
    {
        "cost": 13.4,
        "efficiency": 0.53,
        "name": "gasfiredbig2",
        "pmax": 460,
        "pmin": 100,
        "type": "gasfired"
    },
    {
        "cost": 50.8,
        "efficiency": 0.3,
        "name": "tj1",
        "pmax": 16,
        "pmin": 0,
        "type": "turbojet"
  